** What I want to do:** Having 5 subjects in validtion set resulted in lower accuracy because I'm guessing we simply didnt have enough data to train with. Increasing training size might result in better accuracy.

I'll rerun all the steps, but save weights under different names.

## Step 1: Reset train and test data

In [1]:
%cd data/

/home/ubuntu/distracted_driver/data


In [2]:
!rm -rf train/
!rm -rf test/
!rm -rf valid/
!rm -rf sample/

In [4]:
#!unzip imgs.zip

In [5]:
!ls train/
!ls test/ | wc -l

c0  c1	c2  c3	c4  c5	c6  c7	c8  c9
79726


## Step 2 - Split images into training, test, validation and sample sets

In [6]:
# get image list
import pandas as pd
df_imglist = pd.read_csv('driver_imgs_list.csv')

In [10]:
# copy 3 subjects to validation
import numpy as np
import os,sys,shutil
from numpy import random
from numpy.random import RandomState
ddrandom = RandomState(42)
validation_subjects = ddrandom.choice(df_imglist.subject.unique(),size=3)

In [11]:
# first create validation directory if does not exist
if not os.path.isdir('valid'):
    !mkdir valid

for distraction_classes in os.listdir('train'):
    # create class sub-directory if does not exist
    if not os.path.isdir(os.path.join('valid',distraction_classes)):
        temp_filename = os.path.join('valid',distraction_classes)
        !mkdir $temp_filename
    
    # perform a check for each image in training class
    img_move_counter = 0
    for images in os.listdir(os.path.join('train',distraction_classes)):
        if (images==df_imglist[df_imglist.subject.isin(validation_subjects)]['img']).any():
            shutil.move(os.path.join('train',distraction_classes,images), 
                        os.path.join('valid',distraction_classes,images))
            img_move_counter+=1
    
    # print completion status after each class
    print ('Moved {} images in class {}:'.format(str(img_move_counter),distraction_classes))

Moved 254 images in class c7:
Moved 263 images in class c8:
Moved 292 images in class c6:
Moved 290 images in class c0:
Moved 289 images in class c2:
Moved 294 images in class c4:
Moved 289 images in class c9:
Moved 300 images in class c1:
Moved 293 images in class c3:
Moved 298 images in class c5:


In [12]:
# Copy 1% of training and validation data into sample training and validation set
# first create sample directory if does not exist
if not os.path.isdir('sample'):
    !mkdir sample
    
for directories in ['train','valid']:
    for distraction_classes in os.listdir(directories):
        
        # initialise number of samples and filenames for copying
        number_of_samples = int(0.01*len(os.listdir(os.path.join(directories,distraction_classes))))
        all_filenames = os.listdir(os.path.join(directories,distraction_classes))
        img_cp_counter=0
        
        # run loop for copying
        for images in ddrandom.choice(all_filenames,number_of_samples):
            
            # create sample directory and sub-directory to put image into if does not exist
            if not os.path.isdir(os.path.join('sample',directories)):
                temp_filename = os.path.join('sample',directories)
                !mkdir $temp_filename            
            if not os.path.isdir(os.path.join('sample',directories,distraction_classes)):
                temp_filename = os.path.join('sample',directories,distraction_classes)
                !mkdir $temp_filename
            
            shutil.copy2(os.path.join(directories,distraction_classes,images),
                        os.path.join('sample',directories,distraction_classes,images))
            
            img_cp_counter+=1
        
        # print completion status after each class
        print ('Copied {} images to sample in class {}:'.format(str(img_cp_counter),directories+'/'+distraction_classes))

Copied 17 images to sample in class train/c7:
Copied 16 images to sample in class train/c8:
Copied 20 images to sample in class train/c6:
Copied 21 images to sample in class train/c0:
Copied 20 images to sample in class train/c2:
Copied 20 images to sample in class train/c4:
Copied 18 images to sample in class train/c9:
Copied 19 images to sample in class train/c1:
Copied 20 images to sample in class train/c3:
Copied 20 images to sample in class train/c5:
Copied 2 images to sample in class valid/c7:
Copied 2 images to sample in class valid/c8:
Copied 2 images to sample in class valid/c6:
Copied 2 images to sample in class valid/c0:
Copied 2 images to sample in class valid/c2:
Copied 2 images to sample in class valid/c4:
Copied 2 images to sample in class valid/c9:
Copied 3 images to sample in class valid/c1:
Copied 2 images to sample in class valid/c3:
Copied 2 images to sample in class valid/c5:


## Step 3: Train vgg model